In [2]:
from imports import *

In [3]:
class WhBase:
    
    def __init__(self,driver):
        self.driver = driver
        self.wait = WebDriverWait(self.driver, 5)
        self.ac = ActionChains(self.driver)
        print(self.driver.title)
    
    def xpelem(self, xpt):
        try: return self.wait.until(EC.presence_of_element_located((By.XPATH, xpt)))
        except: return None
    
    def xptext(self, xpt):
        try: return self.driver.find_element(By.XPATH, xpt).text
        except: return None
        
    def xpget(self, xpt):
        try: return self.driver.find_element(By.XPATH, xpt)
        except: return None
    
    def move_click(self, xpt):
        try: self.ac.move_to_element(self.xpget(xp1)).click().perform()
        except: pass
        
    def hover_click(self, xp1, xp2):
        try: self.ac.move_to_element(self.xpget(xp1)).click(self.xpelem(xp2)).perform()
        except: pass
        
    def click_write(self, xpt, text):
        xp = self.xpelem(xpt) if type(xpt) is str else xpt
        if xp is None: return None
        xp.clear()
        if chr(10) not in text:
            self.ac.move_to_element(xp).click().send_keys(text).send_keys(Keys.RETURN).perform()
        else:
            xx = text.split(chr(10))
            for line in xx:
                self.ac.send_keys(line).perform()
                self.ac.key_down(Keys.SHIFT).key_down(Keys.ENTER).key_up(Keys.SHIFT).key_up(Keys.ENTER).perform()
            self.ac.send_keys(Keys.RETURN).perform()
    
    def paste_text(self, xp, text):
        self.ac.send_keys(text).perform()
        
    def sidepane_scroll(self, initial = 1, scrolls=35):
        for i in range(1, scrolls):
            self.driver.execute_script("document.getElementById('pane-side').scrollTop={}".format(initial))
            initial = initial + i + 1
        return initial

    def scroll_by_xpath(self, xpt="(//div[@id='main']//div[@class='ItfyB _3nbHh'])[2]"):
        try:
            xp = self.wait.until(EC.presence_of_element_located((By.XPATH, xpt)))
            self.driver.execute_script("arguments[0].scrollIntoView(true);", xp)
            return 1
        except:
            print('WhBase.scroll_by_xpath(xpt) failed to scroll')
            return 0
            
    def action(self, xpt, cmd='click', text=None):
        xp = self.xpelem(xpt) if type(xpt) is str else xpt
        if xp is None:
            print('xpath identification failure')
            return None
        if cmd =='click':
            self.ac.move_to_element(xp).click().perform()
        elif cmd == 'hover':
            self.ac.move_to_element(xp).perform()
        elif cmd == 'paste' and text is not None:
            self.ac.move_to_element(xp).click().perform()
            xp.clear()
            self.ac.send_keys(text).perform()
        elif cmd == 'select_paste' and text is not None:
            self.click_write(xp, text)
        else:
            self.ac.move_to_element(xp).perform()
        return 1
    
    def panechat(self, n, base, dc=defaultdict()):
        bi = base + '[' + str(n) + ']'
        if self.xpelem(bi) is None: return None
        if len(dc) == 0:
            dc = {'chat_name' : bi + "//div[@class='_21S-L']",
                  'last_text' : bi + "//div[@class='vQ0w7']/span[1]/span[3]",
                  'last_sender' : bi + "//div[@class='vQ0w7']/span[1]/span[1]",
                  'last_datetime' : bi + "/div[@class='y_sn4']/div[@class='Dvjym']",
                  'new_msg_notif' : bi + "//div[@class='Dvjym']//div[@class='_1pJ9J']/span"}
        for k, v in dc.items():
            vl = [v] if type(v) is str else v
            dc[k] = [self.xptext(j) for j in vl]
        return dc        
    
    def msg_sender_dttm(self, n, base):
        xpt = base + "[" + str(n) + "]"
        html = self.wait.until(EC.presence_of_element_located((By.XPATH, xpt))).get_attribute('innerHTML')
        sp = BeautifulSoup(html, "lxml")
        x = sp.find_all('div')
        for i in x:
            try:
                st = i['data-pre-plain-text']
                break
            except:pass
        else: return None, None
        try:
            split_point = st.find(']')
            sender = st[split_point+1:].replace(':','').strip(' ')
            dttm = parse(st.upper()[1:split_point])
            return dttm.strftime("%Y-%m-%d %H:%M"), sender
        except: None, None

    def tearDown(self):
        self.driver.close()
        
    @property
    def get_driver(self):
        return self.driver

In [4]:
def dict_update(cond, ky, vl):
    '''update dictionary values if key already exist otherwise add new into dictionary, single procedure once'''
    if type(vl) is list:
        cond[ky] = vl if not list(cond) else cond.get(ky, []) + vl
    else:
        cond[ky] = [vl] if not list(cond) else cond.get(ky, []) + [vl]
    return cond


def update_key_value(a, b):
    '''updating key & values (values will be appened for existing key)'''
    c = {}
    tls = lambda ls : ls if type(ls) is list else [ls]
    for k, v in a.items():
        c[k] = tls(a[k]) + tls(b[k]) if k in list(b) else tls(a[k])
    else: return {**b, **c}

In [13]:


        
        
    

pane_search = "/html/body/div[1]/div/div/div[3]/div/div[1]/div/div/div[2]/div/div[2]"
body_chat_search_btn = "//div[@id='main']//div[@class='_3ndVb']"
body_chat_name = "//div[@id='main']//div[@class='_21nHd']"
body_text_area = "//footer[@class='_3E8Fg']//p"
body_msg_nav_arrow = '/html/body/div[1]/div/div/div[4]/div/div[2]/div/div[1]/span/div'




class WhatsApp(WhBase):

    body_chat_search_btn = "//div[@id='main']//div[@class='_3ndVb']"
    body_chat_name = "//div[@id='main']//div[@class='_21nHd']"
    body_text_area = "//footer[@class='_3E8Fg']//p"
    body_msg_nav_arrow = '/html/body/div[1]/div/div/div[4]/div/div[2]/div/div[1]/span/div'
    
    
    body_msg_base="(//div[@id='main']//div[@data-testid='msg-container'])"
    msg_sender = "self.msg_sender_dttm"
    sender_text = "//div[@class='_21Ahp']/span[1]/span"
    quoted_sender = "//div[@class='_3pMOs yKTUI']//div[1]/span"
    quoted_text = "//div[@class='_3pMOs yKTUI']//div[2]/span"
    msg_datetime = "self.msg_sender_dttm"
    read_more_btn = "//div[@role='button'][contains(.,'Read more')]"
    
    
    pane_search = "/html/body/div[1]/div/div/div[3]/div/div[1]/div/div/div[2]/div/div[2]"
    pane_base = "(//div[@id='pane-side']//div[@class='_8nE1Y'])"
    pane_chat_name= "//div[@class='_21S-L']"
    pane_last_msg ="//div[@class='vQ0w7']//span[1]/span[3]"
    pane_last_msg_sender ="//div[@class='vQ0w7']//span[1]/span[1]"
    pane_last_msg_datetime = "/div[@class='y_sn4']/div[@class='Dvjym']"
    pane_new_msg_notif = "//div[@class='Dvjym']//div[@class='_1pJ9J']/span"
    
    pnitem = {'pane_chat_name' : "//div[@class='_21S-L']",
        'last_text' : "//div[@class='vQ0w7']/span[1]/span[3]",
        'last_sender' : "//div[@class='vQ0w7']/span[1]/span[1]",
        'last_msg_time' : "/div[@class='y_sn4']/div[@class='Dvjym']",
        'new_msg_notif' : "//div[@class='Dvjym']//div[@class='_1pJ9J']/span"}
    
    def __init__(self):
        chrome_options = Options()
        chrome_options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
        self.d = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
        self.read_store = defaultdict()
        self.store_msg = pd.DataFrame([])
        super().__init__(self.d)
    
    def select_chat(self, chat_name):
        if chat_name is not None and chat_name != '':
            x = self.action(xpt= self.pane_search, cmd='select_paste', text=chat_name)
            y = self.xptext(self.body_chat_name)
            if x is not None and y is not None and chat_name.lower() in y.lower():
                print('chat selection successful')
                return y
            else: 0
        else: return None
        
    def send_msg(self, chat_name, msg):
        x = select_chat
        print('selected_chat returns: ', x)
        if x is not None or x != 0:
            x = self.action(xpt=self.body_text_area, cmd='select_paste', text=msg)
        return self
    
    def msginx(self, n, chatname=None):
        bi = self.body_msg_base + '[' + str(n) + ']'
        if self.xpelem(bi) is None: return None
        self.move_click(bi + self.read_more_btn)
        dttm, sender = self.msg_sender_dttm(n, self.body_msg_base)
        dc = {'chat':chatname,
              'sender': sender,
              'time': dttm,
              'msg': self.xptext(bi + self.sender_text),
              'q_sender':self.xptext(bi + self.quoted_sender),
              'q_text': self.xptext(bi + self.quoted_text),
              'insertime': dt.datetime.now().strftime('%Y-%m-%d %H:%M')
             }
        return dc
    
    def read_msg(self, chat_name, no_of_msg):
        selected_chat_name = self.select_chat(chat_name)
        print('selected_chat returns: ', selected_chat_name)
        if x is not None or x != 0:
            avl = False
            x.move_click(self.body_msg_nav_arrow)
            tm.sleep(0.5)
            for i in range(no_of_msg):
                #y = self.xpelem(self.body_msg_base + '[' + str(no_of_msg) + ']')
                y = None
                while(y is None and avl==False):
                    self.scroll_by_xpath(self.body_msg_base + "[1]")
                    y = self.xpelem(self.body_msg_base + '[' + str(no_of_msg) + ']')
                avl = True
                print(i,':', self.msginx(i))
                print('----------------')
                
    def storing_msg(self, ls):
        if ls is not None and len(ls)>1:
            self.store_msg.loc[len(self.store_msg.index)] = ls
            
    def pane_item(self, n):
        bi = self.pane_base + '[' + str(n) + ']'
        if self.xpelem(bi) is None: return None
        else:
            pane_chat_name = self.xptext(bi + self.pane_chat_name)
            pane_last_msg = self.xptext(bi + self.pane_last_msg)
            pane_last_msg_sender = self.xptext(bi + self.pane_last_msg_sender)
            pane_last_msg_datetime = self.xptext(bi + self.pane_last_msg_datetime)
            pane_new_msg_notif = self.xptext(bi + self.pane_new_msg_notif)
            return [pane_chat_name, pane_last_msg_sender ,pane_last_msg, pane_last_msg_datetime, pane_new_msg_notif]

    def paneitr(self, search=None):
        if search is not None:
            self.select_chat(search)
        ls, lx = [], []
        self.store_msg = pd.DataFrame(columns=['pane_chat_name','pane_last_msg','pane_last_msg_sender','pane_last_msg_datetime','pane_new_msg_notif','insert_dttm','chat_name'])
        x, nonecount, init, i, whilecount = None, 0, 0, 1, 0
        while (x is None or nonecount<4):
            x = self.pane_item(i)
            if x is None: 
                init = self.sidepane_scroll(init)
                i,nonecount = 1, nonecount + 1
            else:
                i = i + 1
                xs = chr(10).join(map(str,x))
                if xs not in ls:
                    ls.append(xs)
                    self.storing_msg(x)
                    whilecount = whilecount + 1
                    nonecount = 0
                    print(whilecount,xs)
                    print('---------------------')
                else:
                    print('no count:', nonecount,'already stored')
        print('out of while')
        self.store_msg.to_csv('pane_' + dt.datetime.now().strftime('%d-%M-%Y_%H%M') + '.csv')
            
        
    def seach_in_pane(self, text):
        pass
    def search_in_group(self, text):
        pass
    def find_forward(self):
        pass
    def find_reply(self):
        pass
    def pending_read(self):
        pass
    def monitor_chat(self):
        pass
    def find_tree(self):
        pass

SyntaxError: invalid syntax (2756602174.py, line 12)

In [10]:
len([])

0

In [11]:
x = WhatsApp()
d = x.get_driver

(25) WhatsApp


In [12]:
x.paneitr('pbsdr10')

ValueError: cannot set a row with mismatched columns

In [ ]:
def pane_read(init=0, storels=[], stop_counter=0, stop_at=3, status=True):
    ls = []
    panediv = "(//div[@id='pane-side']//div[@class='_8nE1Y'])"
    if stop_coudnter>=stop_at or status==False:
        status = False
        return storels
    for n in range(1,50):
        x = x.xpelem(panediv + '[' + str(n) + "]//div[@class='_21S-L']")
        if x is None:
            init = x.sidepane_scroll_by_id(init)
            ln1 = len(storels)
            stx = set(storels + ls)
            print('len: ',len(stx), ' added:', len(stx)-ln1, ' from: ', len(ls))
            if (len(stx)-ln1) == 0: 
                stop_counter=stop_counter + 1
            print('stop_counter:',stop_counter)
            return pane_read(init, list(stx), stop_counter)
        else:
            ls.append(x.text)
    return list(stx)

In [ ]:
#x.pane_read()

In [ ]:
def pane_scroll(driver, initial = 1, scrolls=32):
    for i in range(0, scrolls):
        driver.execute_script("document.getElementById('pane-side').scrollTop={}".format(initial))
        initial += i
    return initial

i = 1

In [ ]:
#i = pane_scroll(d, i)

In [ ]:
#i = pane_scroll(d, i)

In [ ]:
#!i = pane_scroll(d, i)

In [ ]:
#i = pane_scroll(d, i)

In [ ]:
#x.msginx(8)

In [ ]:
#x.msginx(7)

In [ ]:
#x.msginx(9)

In [ ]:
#x.read_msg('emer', 30)

In [ ]:
#for i in range(50, 2, -1):
#    print(i)

In [ ]:
#x.scroll_by_xpath()

In [ ]:
class WaFetch:
    
    def __init__(self, driver):
        self.driver = driver
        self.wait = WebDriverWait(self.driver, 5)
        
    def xptx(self, xpt):
        try: return self.wait.until(EC.presence_of_element_located((By.XPATH, xpt))).text
        except: return None
        
    def xppr(self, xpt):
        try: return self.wait.until(EC.presence_of_element_located((By.XPATH, xpt))).text
        except: return None
    
    def msg_sender_dttm(self,n, base="(//div[@id='main']//div[@class='ItfyB _3nbHh'])"):
        xpt = base + "[" + str(n) + "]"
        html = self.driver.find_element(By.XPATH, xpt).get_attribute('innerHTML')
        sp = BeautifulSoup(html, "lxml")
        x = sp.find_all('div')
        for i in x:
            try:
                st = i['data-pre-plain-text'].upper()
                break
            except:pass
        else: return None, None
        try:
            split_point = st.find(']')
            sender = st[split_point+1:].replace(':','').strip(' ')
            dttm = parse(st[1:split_point].upper())
            return dttm.strftime("%Y-%m-%d %H:%M"), sender
        except: None, None

    def msg(self, n=2, base="(//div[@id='main']//div[@class='ItfyB _3nbHh'])", dc=defaultdict(), fn=None, waitsec=3):
        bi = base + '[' + str(n) + ']'
        if xp(bi) is None: return None
        else:
            dttm, sender = msg_sender_dttm(driver, n)
            dc = {'dttm' : dttm,
                  'sender': sender,
                  'msg' : [self.xp(bi + "//div[@class='_21Ahp']/span[1]/span"),
                           xp(bi + "//div[@class='_27K43 _31p5Q']//span/span")],
                  'quoted_sender' : xp(bi + "//div[@class='_3pMOs yKTUI']//div[1]/span"),
                  'quoted_msg' : xp(bi + "//div[@class='_3pMOs yKTUI']//div[2]/span")}
            return dc

    def chat(driver, n=2, base="(//div[@id='pane-side']//div[@class='_8nE1Y'])", dc=defaultdict(), waitsec=3):
        bi = base + '[' + str(n) + ']'
        if xp(bi) is None: return None
        if len(dc) == 0:
            dc = {'chat_name' : bi + "//div[@class='_21S-L']",
                  'last_text' : bi + "//div[@class='vQ0w7']/span[1]/span[3]",
                  'last_sender' : bi + "//div[@class='vQ0w7']/span[1]/span[1]",
                  'last_msg_time' : bi + "/div[@class='y_sn4']/div[@class='Dvjym']",
                  'new_msg_notif' : bi + "//div[@class='Dvjym']//div[@class='_1pJ9J']/span"}
        for k, v in dc.items():
            vl = [v] if type(v) is str else v
            dc[k] = [xp(j) for j in vl]
        return dc

In [ ]:
#print(chat(d, 2))

In [ ]:
#msg(d, 8)

In [ ]:
#d.refresh()

In [ ]:
class Action:
    
    def __init__(self, driver):
        self.d = driver
        self.w = WebDriverWait(driver, 3)
        self.ac = ActionChains(self.driver)
        
    def action_click(driver, xpt, text):
        elm = self.d.find_element(By.XPATH, xpt) if type(xpt) is str else xpt
        elm.clear()
        ac.move_to_element(elm).send_keys(text).send_keys(Keys.RETURN).perform()

    def action_hover_click(driver, xpt1, xpt2=None):
        elm = self.d.find_element(By.XPATH, xpt1) if type(xpt1) is str else xpt1
        ActionChains(driver).move_to_element(elm).perform()
        if xpt2 is not None:
            elm = wait.until(EC.presence_of_element_located((By.XPATH, xpt2))) if type(xpt2) is str else xpt2
            ActionChains(driver).move_to_element(elm).send_keys(Keys.RETURN).perform()

In [ ]:
def element_waited(driver, xpt, waitsec=5):
    wait = WebDriverWait(driver, waitsec)
    try: return wait.until(EC.presence_of_element_located((By.XPATH, xpt)))
    except: return None

In [ ]:
def element(driver, xpt):
    try: return driver.find_element(By.XPATH, xpt)
    except: return None

In [ ]:
def attr(driver, xpt, value='innerHTML'):
    try: return driver.find_element(By.XPATH, xpt).get_attribute('innerHTML')
    except: return None


#attr(d,"(//div[@id='main']//div[@class='ItfyB _3nbHh'])[5]")

In [ ]:
def option_check(driver, xpt):
    element = driver.find_element(By.XPATH, xpt)
    all_options = element.find_elements(By.TAG_NAME, "option")
    for option in all_options:
        print("Value is: %s" % option.get_attribute("value"))

In [ ]:
def itrdc(xptdc:dict, fn):
    for k, v in xptdc.copy().items():
        ls, lv = [], [v] if type(v) is str else v
        for i in lv:
            ls.append(fn(i))
        else:
            xptdc[k] = ls
    else:
        return xptdc

In [ ]:
def bs_div_attr(html, search_attr='data-pre-plain-text',tag="div"):
    sp = BeautifulSoup(html, "lxml")
    x = sp.find_all(tag)
    for i in x:
        try:
            st = i[search_attr]
            break
        except:pass
    else: return None, None
    try:
        split_point = st.find(']')
        sender = st[split_point+1:].replace(':','').strip(' ')
        dttm = parse(st[1:split_point].upper())
        return dttm.strftime("%Y-%m-%d %H:%M"), sender
    except: None, None

In [ ]:
def xptext(wait, xpt, wsec=1):
    wait = WebDriverWait(driver, wsec)
    try: return wait.until(EC.presence_of_element_located((By.XPATH, xpt))).text
    except: return None
    
def xp(driver, xpt, wsec=3):
    try: return wait.until(EC.presence_of_element_located((By.XPATH, xpt)))
    except: return None

def itemsText(driver, n=2, base="(//div[@id='pane-side']//div[@class='_8nE1Y'])", dc=defaultdict(), fn=None, waitsec=3):
    def xp(xpt):
        try: return wait.until(EC.presence_of_element_located((By.XPATH, xpt))).text
        except: return None
    bi = base + '[' + str(n) + ']'
    if xp(bi) is None: return None
    if len(dc) == 0:
        dc = {'chat_name' : bi + "//div[@class='_21S-L']",
              'last_text' : bi + "//div[@class='vQ0w7']/span[1]/span[3]",
              'last_sender' : bi + "//div[@class='vQ0w7']/span[1]/span[1]",
              'last_msg_time' : bi + "/div[@class='y_sn4']/div[@class='Dvjym']",
              'new_msg_notif' : bi + "//div[@class='Dvjym']//div[@class='_1pJ9J']/span"}
    f = fn if fn is not None else xp
    for k, v in dc.items():
        v = [v] if type(v) is str else v
        dc[k] = [f(j) for j in v]
    return dc

In [ ]:


def pane_index(n, base= "(//div[@id='pane-side']//div[@class='_8nE1Y'])"):
    bi = base + '[' + str(n) + ']'
    dc = {'chat_name' : bi + "//div[@class='_21S-L']",
      'last_text' : bi + "//div[@class='vQ0w7']/span[1]/span[3]",
      'last_sender' : bi + "//div[@class='vQ0w7']/span[1]/span[1]",
      'last_msg_time' : bi + "/div[@class='y_sn4']/div[@class='Dvjym']",
      'new_msg_notif' : bi + "//div[@class='Dvjym']//div[@class='_1pJ9J']/span"}
    return dc

In [ ]:
#msg = itemsText(d)

In [ ]:
#msg(3)

In [ ]:
#wa_msg_items + "[last()]" + "//span[contains(text(),'ERT')]"

In [ ]:
#wa_msg_sender_dttm(d)

In [ ]:
#driver.find_element(By.XPATH, xpt).get_attribute('innerHTML')

In [ ]:
#//*[contains(concat( " ", @class, " " ), concat( " ", "_21Ahp", " " ))]

In [ ]:
#//*[contains(concat( " ", @class, " " ), concat( " ", "_31p5Q", " " ))] | //*[contains(concat( " ", @class, " " ), concat( " ", "_21Ahp", " " ))]

In [ ]:
#//*[contains(concat( " ", @class, " " ), concat( " ", "_1yJoL", " " ))]

In [ ]:
#!//*[contains(concat( " ", @class, " " ), concat( " ", "_31p5Q", " " ))]

In [ ]:
#//*[contains(concat( " ", @text(), " " ), concat( " ", "restored by reset", " " ))]

In [ ]:
#element(d, wa_msg_items + "[last()]").text7

In [ ]:
#element(d, wa_pane_items + "[last()]").text
#ActionChains(d).move_to_element(d.find_element(By.XPATH, wa_pane_search)).click().perform()
#ActionChains(d).send_keys('onami').send_keys(Keys.RETURN).perform()

In [ ]:
HT = element(d,"(//div[@class='_1-lf9 _3mSPV _18q-J'])[last()]").get_attribute('innerHTML')
HT

In [ ]:
#wa_msg_items + "[last()]"

In [ ]:
#sidepane_item(d, n=5)

In [ ]:
#sidepane_item(d, dc=msg_index(n=5))